In [1]:
import pandas as pd
import numpy as np

#  Carga de datos

In [2]:
auctions = pd.read_csv("auctions.csv", dtype={"ref_type_id": np.uint8, "source_id": np.uint8})
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [ ]:
clicks_dtypes = {'advertiser': np.int8, 'action_id': np.float16, 'source_id': np.int8, 'country_code': 'category',
                 'latitude': np.float16, 'longitude': np.float16, 'carrier_id': np.float16, 'brand': np.float16,
                 'touchX': np.float16, 'touchY': np.float16}
clicks = pd.read_csv('clicks.csv', dtypes = clicks_dtypes, low_memory=False)
clicks["created"] =  pd.to_datetime(clicks["created"], errors = "coerce")

In [ ]:
events = pd.read_csv('events.csv', low_memory=False)
events["date"] =  pd.to_datetime(events["date"], errors = "coerce")

In [2]:
installs = pd.read_csv('installs.csv', low_memory = False)
installs["created"] =  pd.to_datetime(installs["created"], errors = "coerce") 

# Partición de los datos por ventanas

In [5]:
#la fecha final debe ser las 00hs del día siguiente a la fecha final de la ventana
def create_window(df, initial_date, final_date, date_feature_name):
    initial = pd.to_datetime(initial_date)
    final = pd.to_datetime(final_date)
    return df.loc[ (df[date_feature_name] >= initial) \
                  & (df[date_feature_name] < final) ]

In [6]:
auctions_18_20 =  create_window(auctions, "2019-04-18", "2019-04-21", "date")
auctions_19_21 =  create_window(auctions, "2019-04-19", "2019-04-22", "date")
auctions_20_22 =  create_window(auctions, "2019-04-20", "2019-04-23", "date")
auctions_21_23 =  create_window(auctions, "2019-04-21", "2019-04-24", "date")
auctions_22_24 =  create_window(auctions, "2019-04-22", "2019-04-25", "date")

In [ ]:
events_18_20 =  create_window(events, "2019-04-18", "2019-04-21", "date")
events_19_21 =  create_window(events, "2019-04-19", "2019-04-22", "date")
events_20_22 =  create_window(events, "2019-04-20", "2019-04-23", "date")
events_21_23 =  create_window(events, "2019-04-21", "2019-04-24", "date")
events_22_24 =  create_window(events, "2019-04-22", "2019-04-25", "date")

In [ ]:
clicks_18_20 =  create_window(clicks, "2019-04-18", "2019-04-21", "created")
clicks_19_21 =  create_window(clicks, "2019-04-19", "2019-04-22", "created")
clicks_20_22 =  create_window(clicks, "2019-04-20", "2019-04-23", "created")
clicks_21_23 =  create_window(clicks, "2019-04-21", "2019-04-24", "created")
clicks_22_24 =  create_window(clicks, "2019-04-22", "2019-04-25", "created")

In [6]:
installs_18_20 =  create_window(installs, "2019-04-18", "2019-04-21", "created")
installs_19_21 =  create_window(installs, "2019-04-19", "2019-04-22", "created")
installs_20_22 =  create_window(installs, "2019-04-20", "2019-04-23", "created")
installs_21_23 =  create_window(installs, "2019-04-21", "2019-04-24", "created")
installs_22_24 =  create_window(installs, "2019-04-22", "2019-04-25", "created")

# Inicialización de df de features

In [8]:
#le pasamos a esta funcion el dataframe de la ventana a la cual deseemos agregarle features posteriormente
def initialize_trainning_df(df, index_name):
    trainning_df = pd.DataFrame(df[index_name].unique())
    trainning_df.columns = ['ref_hash']
    trainning_df = trainning_df.set_index('ref_hash')
    return trainning_df

In [9]:
example = initialize_trainning_df(installs_18_20, "ref_hash")
example.head()

""
ref_hash
5230323462636548010
5097163995161606833
6328027616411983332
7522785771858684314
7882044913917355073


# Sección para añadir features

In [20]:
def charge_df(csv_name, dict_dtypes = None):
    return pd.read_csv(csv_name, dict_dtypes)

In [14]:
def add_feature(df_features, df_to_join, index_df_features, index_df_to_join, feature_name, fill):
    df_features = df_features.merge(df_to_join[[index_df_to_join, feature_name]].set_index(index_df_to_join), how = 'left',\
                                    left_on = index_df_features, right_on = index_df_to_join).fillna(fill)
    return df_features

Para no tener que volver a computar las operaciones, una vez agregados los features podemos guardar el estado final del df de features

In [13]:
def save_changes(df, file_name):
    df.to_csv(file_name, index = True)

## Auctions

Feature 1 -> Tiempo promedio que tarda en aparecer el usuario en una subasta

Feature 2 -> Cantidad de clicks realizados

## Installs

Feature 1